In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import MDAnalysis as mda
import numpy as np
import matplotlib.pyplot as plt
from chemtrain import quantity
from tqdm import tqdm
from jax import random
from jax import numpy as jnp
import numpy as onp

%config InlineBackend.figure_formats = ['svg']
%matplotlib inline
key = random.PRNGKey(21)
kT = 300. * quantity.kb
state_kwargs = {'kT': kT}

/home/weilong/miniconda3/envs/chemsim/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-04 10:26:41.145775: W external/xla/xla/service/gpu/ir_emitter_unnested.cc:1171] Unable to parse backend config for custom call: Could not convert JSON string to proto: Expected : between key:value pair.
= 0 : ui8, lower = t
^
Fall back to parse the raw backend config str.


In [2]:

# Load the LAMMPS trajectory
u = mda.Universe("/home/weilong/workspace/chemsim-lammps/examples/alan_simulation/alanine_solvated_raw.lmpdat",
                 "/home/weilong/workspace/chemsim-lammps/examples/alan_simulation/alanine_solvated_painn.lammpstrj", 
                 topology_format="DATA",
                 format='LAMMPSDUMP')

# - Molecule 1 (alanine dipeptide) contains atoms with IDs 1-22
# - ACE C (atom ID 5, index 4 in 0-based)
# - ALA N (atom ID 7, index 6)
# - ALA CA (atom ID 9, index 8)
# - ALA C (atom ID 15, index 14)
# - NME N (atom ID 17, index 16)
# ------------------------------------------------------------

# Define phi and psi dihedral groups using atom indices
phi_group = u.atoms[[1, 7, 9, 15]]  # (ACE C, ALA N, ALA CA, ALA C)
psi_group = u.atoms[[7, 9, 15, 17]]  # (ALA N, ALA CA, ALA C, NME N)

phi_angles = []
psi_angles = []

# Calculate dihedrals for each frame
# random selection of 100000 frames

for ts in tqdm(u.trajectory):
    phi = phi_group.dihedral.value()
    psi = psi_group.dihedral.value()
    phi_angles.append(phi)
    psi_angles.append(psi)

# Convert to numpy arrays
phi = np.array(phi_angles)
psi = np.array(psi_angles)


u_ref = mda.Universe("/home/weilong/workspace/chemsim-lammps/examples/alan_simulation/alanine_solvated_raw.lmpdat",
                     "/home/paul/nn_prior/examples/spice/output/SPICE_Alanine_Explicit/alanine_solvated.lammpstrj",
                        topology_format="DATA",
                        format='LAMMPSDUMP')

ref_phi_group = u_ref.atoms[[1, 7, 9, 15]]  # (ACE C, ALA N, ALA CA, ALA C)
ref_psi_group = u_ref.atoms[[7, 9, 15, 17]]  # (ALA N, ALA CA, ALA C, NME N)

ref_phi_angles = []
ref_psi_angles = []

for ts in tqdm(u_ref.trajectory[::10]):
    ref_phi = ref_phi_group.dihedral.value()
    ref_psi = ref_psi_group.dihedral.value()
    ref_phi_angles.append(ref_phi)
    ref_psi_angles.append(ref_psi)

# Convert to numpy arrays
ref_phi = np.array(ref_phi_angles)
ref_psi = np.array(ref_psi_angles)




/home/weilong/miniconda3/envs/chemsim/lib/python3.11/site-packages/MDAnalysis/coordinates/LAMMPS.py:667: UserWarning: Reader has no dt information, set to 1.0 ps
  ts.data['time'] = step_num * ts.dt
100%|██████████| 2568/2568 [00:08<00:00, 309.20it/s]


In [3]:

u_mace = mda.Universe("/home/weilong/workspace/chemsim-lammps/examples/alan_simulation/alanine_solvated_raw.lmpdat",
                     "/home/weilong/workspace/chemsim-lammps/examples/alan_simulation/mace_run/alanine_solvated_mace.lammpstrj",
                        topology_format="DATA",
                        format='LAMMPSDUMP')

mace_phi_group = u_mace.atoms[[1, 7, 9, 15]]  # (ACE C, ALA N, ALA CA, ALA C)
mace_psi_group = u_mace.atoms[[7, 9, 15, 17]]  # (ALA N, ALA CA, ALA C, NME N)

mace_phi_angles = []
mace_psi_angles = []

for ts in tqdm(u_mace.trajectory[::10]):
    mace_phi = mace_phi_group.dihedral.value()
    mace_psi = mace_psi_group.dihedral.value()
    mace_phi_angles.append(mace_phi)
    mace_psi_angles.append(mace_psi)

# Convert to numpy arrays
mace_phi = np.array(mace_phi_angles)
mace_psi = np.array(mace_psi_angles)

100%|██████████| 6464/6464 [00:22<00:00, 292.37it/s]


In [4]:
def plot_histogram_free_energy(ax, phi, psi, kbt, degrees=True, ylabel=False, title=""):
    """Plot 2D free energy histogram for alanine from the dihedral angles."""
    cmap = plt.get_cmap('viridis')

    if degrees:
        phi = jnp.deg2rad(phi)
        psi = jnp.deg2rad(psi)

    h, x_edges, y_edges = jnp.histogram2d(phi, psi, bins=60, density=True)

    h = jnp.log(h) * -(kbt / 4.184)
    x, y = onp.meshgrid(x_edges, y_edges)

    cax = ax.pcolormesh(x, y, h.T, cmap=cmap, vmax=5.25)
    ax.set_xlabel('$\phi$ [rad]')
    if ylabel:
        ax.set_ylabel('$\psi$ [rad]')
    ax.set_title(title)
    
    return ax, cax

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, layout="constrained", figsize=(6, 3), sharey=True)

ax1, _ = plot_histogram_free_energy(ax1, ref_phi, ref_psi, kT, ylabel=True, title="Paul Reference")
ax2, _ = plot_histogram_free_energy(ax2, phi, psi, kT, title="PaiNN")
ax3, cax = plot_histogram_free_energy(ax3, mace_phi, mace_psi, kT, title="MACE")
cbar = fig.colorbar(cax)
cbar.set_label('Free Energy (kcal/mol)')

